# Compare the facilities being offered exemptions from reporting with all USA facilities
Do these facilities really deserve to be trusted to do the right thing?

Compare violations per facility, and also look at inspections per facility to see how often the EPA has deemed it necessary to look into these facilities.

In [ ]:
import pandas as pd

### Run the cell for either Violations per facility or Inspections per facility
Then execute the rest of the cells to end with the chart.

## Violations per facility - CAA

In [ ]:
usa_filename = '/home/steve/workspace/EDGI/ECHO-Cross-Program/CSVs/caa_viol_years.csv'
exemptions_filename = '/home/steve/workspace/EDGI/ECHO-Cross-Program/CSVs/CAA-Violations-Exemptions-FullList.csv'
date_column = 'EARLIEST_FRV_DETERM_DATE'
data_column = 'Viol/fac'
graph_title = 'Violations per facility--All USA CAA facilities versus Exemption facilities'
y_axis = 'Violations/facility'

## Inspections per facility - CAA

In [ ]:
usa_filename = '/home/steve/workspace/EDGI/ECHO-Cross-Program/CSVs/caa_insp_years.csv'
exemptions_filename = '/home/steve/workspace/EDGI/ECHO-Cross-Program/CSVs/CAA-Inspections-Exemptions-FullList.csv'
date_column = 'ACTUAL_END_DATE'
data_column = 'Insp/fac'
graph_title = 'Inspections per facility--All USA CAA facilities versus Exemption facilities'
y_axis = 'Inspections/facility'

## Enforcements per facility - CAA

In [ ]:
usa_filename = '/home/steve/workspace/EDGI/ECHO-Cross-Program/CSVs/caa_enf_count_years.csv'
exemptions_filename = '/home/steve/workspace/EDGI/ECHO-Cross-Program/CSVs/CAA-Penalties-Exemptions-FullList.csv'
date_column = 'SETTLEMENT_ENTERED_DATE'
data_column = 'Enf/fac'
graph_title = 'Enforcements per facility--All USA CAA facilities versus Exemption facilities'
y_axis = 'Enforcements/facility'

In [ ]:
usa = pd.read_csv(usa_filename)
usa.set_index('Year', inplace=True)
usa['Who'] = 'USA'
usa.rename(columns={'USA' : data_column}, inplace=True)
usa

In [ ]:
num_facs = 386
exemptions = pd.read_csv(exemptions_filename)
exemptions = exemptions[['REGISTRY_ID', date_column]]
exemptions['Date'] = pd.to_datetime(exemptions[date_column])
exemptions['Year'] = pd.DatetimeIndex(exemptions['Date']).year
exemptions

In [ ]:
df = exemptions.groupby(['Year', 'REGISTRY_ID']).count()
df = df.reset_index()
df1 = df.groupby(['Year']).count()
df2 = df.groupby(['Year']).sum()
df3 = df2.join(df1, lsuffix='_l', rsuffix='_r')
df3 = df3[['Date_l']]
df3[data_column] = df3['Date_l']/num_facs
df3['Who'] = 'Exemptions'
df3.drop(['Date_l'], axis=1, inplace=True)
df3

In [ ]:
df4 = pd.concat([df3, usa], axis=0)
df4

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#set seaborn plotting aesthetics
sns.set(style='white')

#create grouped bar chart

fig, ax = plt.subplots(figsize=(10,10))
try:
    g = sns.barplot(x='Year', y=data_column, hue='Who', data=df4)
    g.set_title(graph_title)
    ax.set_ylabel(y_axis)
except TypeError as te:
    print("TypeError: {}".format(str(te)))
